# DoD-Design search
## Example-1
### Stowing application

In [2]:
# Imports
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG, clear_output
# import plotly.express as px

from pydrake.all import *
from pydrake.examples.manipulation_station import ManipulationStation

from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import (
    StartMeshcat, MeshcatJointSlidersThatPublish, MeshcatPoseSliders)
from manipulation.scenarios import AddMultibodyTriad, SetColor, AddShape

import pandas as pd

In [3]:
from odio_urdf import *
import math
import sys
# from operator import add
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import copy
import time

In [4]:
meshcat = StartMeshcat()

In [5]:
# Initiate the tree structure with a base
# For now we have the module class within the main class
# This class allows one to define the modules and the relations between them
# Also contains functions to automatically attach modules together
# creating robots. The class is also capable of exporting the robot as a 
# URDF and uploads it to the meshcat server for visualisation
class robotDoD:
    # A class to define all the modules and their description
    class robotModule:
        def __init__(self, name):
            self.name = name;
            # Define interfaces poition and orientation
            self.intA = np.array([[0,0,0],[0,0,0]]) # Interface A origin and orientation
            self.intB = np.array([[0,0,0],[0,0,0]]) # Interface B origin and orientation
            self.mesh_path = "" # Set the location of the path
            self.scale = "0.001 0.001 0.001" # Set scale as string
            self.type = 0 # 0-link; 1-joint    
            self.cost = 0 # Cost associated with a particular module
            # Parent type gives the compatibility between modules
            # Links cannot connect to links and same with joints
            self.parent_type = -1 # -1 if parent type is not set
                         
  
    # A helper class for transformation
    class helperFuncs:
        def __init__(self):
            pass
            
        def list2space(self, com):
            return str(com[0])+' '+str(com[1])+' '+str(com[2])
    
        def rotZ(self, angle):
            return np.array([[np.cos(angle), -np.sin(angle), 0],[np.sin(angle), np.cos(angle), 0],[0,0,1]])

        def rotY(self, angle):
            return np.array([[np.cos(angle), 0, np.sin(angle)],[0, 1, 0],[-np.sin(angle), 0, np.cos(angle)]])

        def rotX(self, angle):
            return np.array([[1,0,0], [0, np.cos(angle), -np.sin(angle)],[0, np.sin(angle), np.cos(angle)]])
    
        def transform_vector(self, axis, angle, vector):
            if axis == 0:
                return self.rotX(angle).dot(vector)
            if axis == 1:
                return self.rotY(angle).dot(vector)
            if axis == 2:
                return self.rotZ(angle).dot(vector)
            else:
                raise AssertionError("Error: Rotation about unknown axis requested") 
                
        # Resolve the axis and angle when aligned
        def resolve_axis(self, module):
            def foo(x):
                return x[0]*x[1]*x[2]

            def non_lin(x):
                return np.linalg.norm(x)

            lin_constraint = LinearConstraint(module.intA[1], 0, 0)
            nonlin_constraint = NonlinearConstraint(non_lin, 1, 1)

            sol = minimize(foo,[1/np.sqrt(3),1/np.sqrt(3),1/np.sqrt(3)], constraints = [lin_constraint, nonlin_constraint])
            return sol['x']            
    
    def __init__(self, name):
        self.name = name
        # Initiate the helper class
        self.helper_funcs = self.helperFuncs()
        # Declare materials
        self.materials = Group(
            Material("blue", Color(rgba="0 0 0.8 1.0")),
            Material("red", Color(rgba="0.8 0 0 1.0")),
            Material("black", Color(rgba="0.0 0.0 0.0 1.0")))  
        self.material_color = ["blue", "red", "black"]
        self.module_types = ["linkMod", "jointMod"]
        # Declare the robot
        self.robot = Robot(self.name, self.materials)
        # Declare robots tree structure
        self.tree = {}
        self.prev_module = []
        # Initate all the modules internally
        self.initiate_modules()
        
    def add_root(self):        
        # Add root module to the robot URDF
        root_mod = self.robotModule('root_mod')
        # Define the module
        root_mod.intA = np.array([[0,0,0],[0,0,-1]])
        root_mod.intB = np.array([[0,0,0.01],[0,0,1]])
        root_mod.type = -1
        
        root_link_urdf = Link(
                    Visual(
                        Origin(rpy = "0 0 0", xyz = self.helper_funcs.list2space(root_mod.intA[0])),
                        Material (name = "black"),
                        Geometry(Cylinder(length = 0.02, radius = 0.05))),
                    name = root_mod.name)
        
        self.prev_module = root_mod.intB
        self.robot(root_link_urdf)
        self.tree[root_mod.name] = [root_mod, root_mod.intB]
    
    def add_module(self, module):
        # Always start from the root module which initates the self.prev_module
        # Check if prev_module is empty => root not initiated 
        if not self.prev_module.any():
            raise AssertionError("Error: Add root module first before initiating the tree")
        # Get info from self.prev_module
        # Compute connection rules from prev and current modules
        current_origin, current_orientation = self.computeConnection(module)
        
        link_name = self.module_types[module.type]+ '_'+str(len(self.tree.keys()))
        # Generate module URDF
        module_link_urdf = Link(
                         Visual(
                            Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space(current_orientation)),
                            Material(name = self.material_color[module.type]),
                            Geometry(Mesh(filename = module.mesh_path, scale = module.scale))),
                        name= link_name)
        
        # Add the module to the robot URDF
        self.robot(module_link_urdf)

        # Now add joints everytime a new module is added
        # If link is added then add a revolute joint to the previous module
        # If a joint is added then add a fixed joint with the previous module
        if module.type == -1:
            # Do nothing if root            
            pass
        
        if module.type == 0: # If link
            joint_type = "revolute"
            joint_name  = 'joint_' + str(len(self.tree.keys()))+'_'+ joint_type
            module_joint_urdf = Joint(
                                    Parent(list(self.tree.keys())[-1]),
                                    Child(link_name),
                                    Origin(xyz = self.helper_funcs.list2space(self.prev_module[0]), rpy = self.helper_funcs.list2space([0,0,0])),
                                    Axis(xyz = self.helper_funcs.list2space(self.prev_module[1])),
                                    type = joint_type,
                                    name= joint_name)

            self.robot(module_joint_urdf)
            
            # If revolute then also add an actuator here
            actuator_name = 'act_'+joint_name
            module_actuator_urdf = Transmission(Type("SimpleTransmission"),
                                    Actuator(Mechanicalreduction('1'), name = actuator_name),
                                    Transjoint(joint_name),
                                    name = actuator_name)

            self.robot(module_actuator_urdf)
            
            
        if module.type == 1 and list(self.tree.keys())[-1] != "root_mod": # If joint
            joint_type = "fixed"
            joint_name  = 'joint_' + str(len(self.tree.keys()))+'_'+ joint_type
            module_joint_urdf = Joint(
                                    Parent(list(self.tree.keys())[-1]),
                                    Child(link_name),
                                    Origin(xyz = self.helper_funcs.list2space(self.prev_module[0]), rpy = self.helper_funcs.list2space([0,0,0])),
                                    Axis(xyz = self.helper_funcs.list2space(self.prev_module[1])),
                                    type = joint_type,
                                    name= joint_name)
            
            self.robot(module_joint_urdf)
        
        if module.type == 1 and list(self.tree.keys())[-1] == "root_mod": # If joint and previous is root
            joint_type = "revolute"
            joint_name  = 'joint_' + str(len(self.tree.keys()))+'_'+ joint_type
            module_joint_urdf = Joint(
                                    Parent(list(self.tree.keys())[-1]),
                                    Child(link_name),
                                    Origin(xyz = self.helper_funcs.list2space(self.prev_module[0]), rpy = self.helper_funcs.list2space([0,0,0])),
                                    Axis(xyz = self.helper_funcs.list2space(self.prev_module[1])),
                                    type = joint_type,
                                    name= joint_name)

            self.robot(module_joint_urdf)
            
            # If continuous then also add an actuator here
            actuator_name = 'act_'+joint_name
            module_actuator_urdf = Transmission(Type("SimpleTransmission"),
                                    Actuator(Mechanicalreduction('1'), name = actuator_name),
                                    Transjoint(joint_name),
                                    name = actuator_name)
            self.robot(module_actuator_urdf)
        
        # Save all the origins and orientations in a tree
        self.prev_module = self.new_prev_module
        self.tree[link_name] = [module, self.prev_module]

        
    def add_eef(self):
        eef_link_name = 'eef'
        # Generate module URDF
        eef_link_urdf = Link(name= eef_link_name)
        # Add the eef link to the robot URDF
        self.robot(eef_link_urdf)
        # Now add the eef to the end link
        joint_type = "fixed"
        joint_name  = 'eef_joint'
        eef_joint_urdf = Joint(
                        Parent(list(self.tree.keys())[-1]),
                        Child(eef_link_name),
                        Origin(xyz = self.helper_funcs.list2space(self.prev_module[0]), rpy = self.helper_funcs.list2space([0,0,0])),
                        Axis(xyz = self.helper_funcs.list2space(self.prev_module[1])),
                        type = joint_type,
                        name= joint_name)

        self.robot(eef_joint_urdf)

        
    def constructURDF(self, path = None):
        name = self.robot.name
        if path != None:
            name = path+name
        
        name = name+'.urdf'
        # Writing robot URDF to path         
        with open(name, 'w') as f:
            print(self.robot, file=f)
            print('Robot design successfully saved as '+name)
    
    def visualiseRobot(self, urdf_path, meshcat, axis=False):
        # Loading the updated robot URDF
        self.builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step=0.001)
        self.model = Parser(self.plant).AddModelFromFile(urdf_path)
        # Adding a sphere for visualisation       
#         position = self.tree['jointMod_1'][0]
#         position =  [3.0e-02, -3.0e-04,  6.3e-02 ]
#         sphere = AddShape(self.plant, Sphere(0.01), "sphere")
#         X_WO = RigidTransform(position)
#         self.plant.WeldFrames(self.plant.world_frame(), self.plant.GetFrameByName("sphere"), X_WO)
        
        self.plant.Finalize()
        
        if axis == True:
            for body in self.plant.GetBodyIndices(self.model):
                body_name = self.plant.get_body(body).name()
                AddMultibodyTriad(self.plant.GetFrameByName(body_name, self.model), \
                                  self.scene_graph)
        
        meshcat.Delete()
        meshcat.DeleteAddedControls()
        
        self.visualizer = MeshcatVisualizerCpp.AddToBuilder(self.builder, self.scene_graph, meshcat)
        
        self.diagram = self.builder.Build()
        self.diagram_context = self.diagram.CreateDefaultContext()
        self.diagram.Publish(self.diagram_context)
        
    def computeConnection(self, module):
        current_orientation = np.zeros(3)
        # Compute the rotation axis
        temp = np.cross(self.prev_module[1], module.intA[1])
        sinangle = np.linalg.norm(temp)
        cosangle = np.round(np.dot(self.prev_module[1], module.intA[1]))
        # Hangle sinangle being zero
        # print(module.name)
        if sinangle == 0 and cosangle == 1:
            # We need the vector flipped
            # We need to resolve redundancy here by solving an optimisation problem
            rotaxis = self.helper_funcs.resolve_axis(module)
            rotangle = np.pi
            
        if sinangle == 0 and cosangle == -1:
            rotangle = 0
            rotaxis = np.zeros(3)
            
        if sinangle != 0:
            rotaxis = temp/sinangle
            # Compute the rotation angle
            rotangle = np.arctan2(sinangle, cosangle)
        
        # Compute the quaternion
        r = R.from_rotvec(rotangle*rotaxis)
        # Convert to Euler rotation 'xyz'
        current_orientation = r.as_euler('xyz')

        # Rotation matrix
        rmat = r.as_matrix()
        current_origin = np.dot(rmat,(-module.intA[0]))  
        # Next orientation
        next_orientation = np.dot(rmat, module.intB[1])
        next_origin = current_origin + np.dot(rmat, module.intB[0])
        # Update prev_module        
        self.new_prev_module = np.array([next_origin, np.round(next_orientation)])
    
        return current_origin, current_orientation
    
    def visualiseModule(self, module, meshcat):
        # Loading the module
        self.builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step=0.001)
        
        module_urdf = Robot(module.name, self.materials)
        # Generate module URDF
        module_link_urdf = Link(
                         Visual(
                            Origin(xyz = self.helper_funcs.list2space([0,0,0]), rpy = self.helper_funcs.list2space([0,0,0])),
                            Material(name = self.material_color[module.type]),
                            Geometry(Mesh(filename = module.mesh_path, scale = module.scale))),
                        name=module.name)
        
        # Add the module to the robot URDF
        module_urdf(module_link_urdf)
        name = module.name+'.urdf'
        with open(name, 'w') as f:
            print(module_urdf, file=f)
            print('Module URDF successfully saved as '+name)
        
        urdf_path = './'+name
        self.model = Parser(self.plant).AddModelFromFile(urdf_path)
        self.plant.Finalize()

        meshcat.Delete()
        meshcat.DeleteAddedControls()

        self.visualizer = MeshcatVisualizerCpp.AddToBuilder(self.builder, self.scene_graph, meshcat)

        self.diagram = self.builder.Build()
        self.diagram_context = self.diagram.CreateDefaultContext()
        self.diagram.Publish(self.diagram_context)
        
    # One can initiate it from within or externally define new modules that 
    # needs to be considered for the simulation
    def initiate_modules(self):
        # jmod
        self.jmod = self.robotModule('jmod')
        # Define the module
        self.jmod.intA = np.array([[0,-0.0035,0],[0,-1,0]])
        self.jmod.intB = np.array([[0.03, 0.0495, 0.0003],[1,0,0]])
        self.jmod.type = 1
        self.jmod.mesh_path = "./meshes/j_module.obj"
        self.jmod.cost = 100
        self.jmod.parent_type = [-1, 0]
        
        # l1mod
        self.l1mod = self.robotModule('l1mod')
        # Define the module
        self.l1mod.intA = np.array([[0.04,0,0],[1,0,0]])
        # l1mod.intB = np.array([[0.035,0.16,0],[1,0,0]])
        self.l1mod.intB = np.array([[0.04,0.16,0],[1,0,0]])
        self.l1mod.type = 0
        self.l1mod.mesh_path = "./meshes/l1_module.obj"
        self.l1mod.cost = 20
        self.l1mod.parent_type = [-1, 1]

        # l2mod
        self.l2mod = self.robotModule('l2mod')
        # Define the module
        self.l2mod.intA = np.array([[0.04,0,0],[1,0,0]])
        self.l2mod.intB = np.array([[0,0.148,0],[0,1,0]])
        self.l2mod.type = 0
        self.l2mod.mesh_path = "./meshes/l2_module.obj"
        self.l2mod.cost = 20
        self.l2mod.parent_type = [-1, 1]

        # l3mod
        self.l3mod = self.robotModule('l3mod')
        # Define the module
        self.l3mod.intA = np.array([[0,0.148,0],[0,1,0]])
        self.l3mod.intB = np.array([[0.04, 0, 0],[1,0,0]])
        self.l3mod.type = 0
        self.l3mod.mesh_path = "./meshes/l3_module.obj"
        self.l3mod.cost = 20
        self.l3mod.parent_type = [-1, 1]
        
        self.declared_modules = [self.jmod, self.l1mod, self.l2mod, self.l3mod]

In [6]:
# This class is at the intermediate level where it takes in the robot
# module class and uses the compatibility rules from the previous
# class to constructs meaningful robots, this also does all the
# housekeeping of creating multiple instances of the robot environment
# i.e., creating branches and populating the correct branch

# Also contains problem level objects to define the cost and 
# compute optimal design in the given expanded branch
class Node:
    def __init__(self, roboDoD):
        self.depth = 0
        # Dont use self.base_node to prevent editing it by mistake
        self.parent_node = roboDoD
        self.num_modules = len(roboDoD.declared_modules)
        # Directory to save the constructed URDFs
        self.node_path = './node_urdfs/'
    
    def get_compatible_modules(self):
        # Get the type of the last node
        if not self.parent_node.tree:
            raise AssertionError("Error: No modules found in base node") 
        
        parent_type = self.parent_node.tree[list(roboDoD.tree)[-1]][0].type
        # Identify all compatible nodes
#         self.compatible_modules = []
        compatible_modules = []
        for module in roboDoD.declared_modules:
            if parent_type in module.parent_type:
                compatible_modules.append(module)
        return compatible_modules
    
    # Create multiple instances of the parent node
    # If num_instances is not specified then create 
    # as many as the number of modules
    def create_instances(self, num_instances=None):
        if num_instances == None:
            num_instances = self.num_modules
        node_instances = []
        for i in range(num_instances):
            # Makes instances of whatever is present in the parent node
            node_instance = copy.deepcopy(self.parent_node)
            module_name = 'node'+str(self.depth)+str(i)
            # Change instance name
            node_instance.name = module_name
            # Change URDF name
            node_instance.robot.name = module_name
            node_instances.append(node_instance)
            
        return node_instances


    # Expand corresponding nodes of robots based on compatible modules    
    def expand_node(self):
        # Get compatible nodes first
        compatible_modules = self.get_compatible_modules()
        # Now create required number of instances
        node_instances = self.create_instances()
        # Add each module to each new instance
        for i in range(len(compatible_modules)):
            node_instances[i].add_module(compatible_modules[i])
            # Construct the URDF files of these instances
            # Add an eef link at the end
            temp_node_instance = copy.deepcopy(node_instances[i])
            temp_node_instance.add_eef()
            temp_node_instance.constructURDF(self.node_path)
            
        self.child_nodes = node_instances
        # Increase the depth of the tree        
        self.depth += 1

    
    def compute_cost(self):
        # This changes with the requirements and the task
        # THIS IS SPECIFIC FOR EXAMPLE-1
        # This part can be parallelised
#         TOBE CONTINUOUED FROM HERE!!!
        
        
        pass
    
    # Default visualisation is set to False
    def computeOptimalDesign(self, visualise_search = False):
        pass

In [9]:
roboDoD = robotDoD('test')
roboDoD.add_root()
roboDoD.add_module(roboDoD.jmod)
roboDoD.declared_modules

In [10]:
node = Node(roboDoD)
compatible_modules = node.get_compatible_modules()
node_instances = node.create_instances(len(compatible_modules))
node.expand_node()
node.child_nodes

Robot design successfully saved as ./node_urdfs/node00.urdf
Robot design successfully saved as ./node_urdfs/node01.urdf
Robot design successfully saved as ./node_urdfs/node02.urdf


In [15]:
meshcat.Delete()
meshcat.DeleteAddedControls()

In [16]:
# Initiate the robot class
roboDoD = robotDoD('test')
roboDoD.add_root()
roboDoD.add_module(roboDoD.jmod)
roboDoD.declared_modules
# Initiate the node class
node = Node(roboDoD)
compatible_modules = node.get_compatible_modules()
node_instances = node.create_instances(len(compatible_modules))
node.expand_node()
node.child_nodes
# Run search loop until success
end_game = False
while not end_game:
    # Lets start with loading a given node and computing the cost
    builder = DiagramBuilder()
    model_name = node.child_nodes[0].robot.name
    urdf_path = './node_urdfs/'+model_name+'.urdf'
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, model_name)
    # Weld the robot to the world frame
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame())

    # Spawn spherical work marker
    sphere = Parser(plant, scene_graph).AddModelFromFile('./urdfs/helper/sphere.urdf','sphere')
    # We can sample end-points on the surface of the sphere
    cart0 = np.array([-0.3, 0.4, 0.3])
    X_R = RigidTransform(cart0)
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(sphere)[0]).body_frame(), X_R)
    plant.Finalize()

    # Select the last frame as the eef_frame
    gripper_frame = plant.GetBodyByName("eef").body_frame()
#     AddMultibodyTriad(gripper_frame, scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    diagram.Publish(context)
    
    time.sleep(2)
    
    ik = InverseKinematics(plant, plant_context)
    ik.AddPositionConstraint(
                gripper_frame, [0, 0, 0], plant.world_frame(), 
                cart0, cart0)
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    qr = result.GetSolution(ik.q())
    # Print result
    print(result.is_success())
    # Print the residual cost
    print('Best cost: ', result.get_optimal_cost())
    # Visualise the best solution
    plant.SetPositions(plant.GetMyContextFromRoot(context),model,qr)
    diagram.Publish(context)
    
    time.sleep(2)
    
    # So now expanding from the previous node
    # Update parent_node
    node.parent_node = node.child_nodes[0]
    # Expand node
    node.expand_node()
    # Now run the upload and compute cost cells
    end_game = result.is_success()

[2022-01-18 09:53:56.008] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-18 09:53:56.031] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/j_module.obj references 29fd947b-97f8-4696-91a7-40d64dd5beae.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/29fd947b-97f8-4696-91a7-40d64dd5beae.mtl"


Robot design successfully saved as ./node_urdfs/node00.urdf
Robot design successfully saved as ./node_urdfs/node01.urdf
Robot design successfully saved as ./node_urdfs/node02.urdf
False
Best cost:  35.19128977435114


[2022-01-18 09:54:00.087] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-18 09:54:00.099] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-18 09:54:00.120] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/j_module.obj references 29fd947b-97f8-4696-91a7-40d64dd5beae.mtl, 

Robot design successfully saved as ./node_urdfs/node10.urdf
Robot design successfully saved as ./node_urdfs/node11.urdf
Robot design successfully saved as ./node_urdfs/node12.urdf
False
Best cost:  4.184272159301816


[2022-01-18 09:54:04.152] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-18 09:54:04.164] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-18 09:54:04.176] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl,

Robot design successfully saved as ./node_urdfs/node20.urdf
Robot design successfully saved as ./node_urdfs/node21.urdf
Robot design successfully saved as ./node_urdfs/node22.urdf
False
Best cost:  6.781329839059292


[2022-01-18 09:54:08.237] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-18 09:54:08.247] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-18 09:54:08.260] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl,

Robot design successfully saved as ./node_urdfs/node30.urdf
Robot design successfully saved as ./node_urdfs/node31.urdf
Robot design successfully saved as ./node_urdfs/node32.urdf
True
Best cost:  1.6057197398397076
Robot design successfully saved as ./node_urdfs/node40.urdf
Robot design successfully saved as ./node_urdfs/node41.urdf
Robot design successfully saved as ./node_urdfs/node42.urdf


In [9]:
# Lets start with loading a given node and computing the cost
builder = DiagramBuilder()
model_name = node.child_nodes[0].robot.name
urdf_path = './node_urdfs/'+model_name+'.urdf'
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, model_name)
# Weld the robot to the world frame
plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame())

# Spawn spherical work marker
sphere = Parser(plant, scene_graph).AddModelFromFile('./urdfs/helper/sphere.urdf','sphere')
# We can sample end-points on the surface of the sphere
cart0 = np.array([-0.3, 0.4, 0.3])
X_R = RigidTransform(cart0)
plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(sphere)[0]).body_frame(), X_R)
plant.Finalize()

# Select the last frame as the eef_frame
gripper_frame = plant.GetBodyByName("eef").body_frame()
AddMultibodyTriad(gripper_frame, scene_graph)

meshcat.Delete()
meshcat.DeleteAddedControls()

visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)

q0 = plant.GetPositions(plant_context)
diagram.Publish(context)

[2022-01-13 15:27:18.058] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-13 15:27:18.073] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl, but Meshcat could not open filename "/home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl"
[2022-01-13 15:27:18.090] [console] [warning] Meshcat: Failed to load texture. /home/nightmareforev/git/research/lcl-robots/src/tools-test-py/node_urdfs/meshes/l1_module.obj references f383e0f4-56f7-4c7b-a1d1-cbc3f41bfe7b.mtl,

In [10]:
ik = InverseKinematics(plant, plant_context)
ik.AddPositionConstraint(
            gripper_frame, [0, 0, 0], plant.world_frame(), 
            cart0, cart0)
prog = ik.get_mutable_prog()
q = ik.q()
prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
prog.SetInitialGuess(q, q0)
result = Solve(ik.prog())
qr = result.GetSolution(ik.q())
# Print result
print(result.is_success())
# Print the residual cost
print('Best cost: ', result.get_optimal_cost())
# Visualise the best solution
plant.SetPositions(plant.GetMyContextFromRoot(context),model,qr)
diagram.Publish(context)

True
Best cost:  0.6268888563811539


In [11]:
# So now expanding from the previous node
# Update parent_node
node.parent_node = node.child_nodes[0]
# Expand node
node.expand_node()
# Now run the upload and compute cost cells

Robot design successfully saved as ./node_urdfs/node50.urdf
Robot design successfully saved as ./node_urdfs/node51.urdf
Robot design successfully saved as ./node_urdfs/node52.urdf


In [12]:
meshcat.Delete()
meshcat.DeleteAddedControls()